In [2]:
import tensorflow as tf
import numpy as np

gpu_options = tf.GPUOptions(visible_device_list="3")
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True, device_count={'GPU': 1}, gpu_options=gpu_options)
config.gpu_options.per_process_gpu_memory_fraction = 0.3
sess = tf.Session(config=config)

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [5]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f57641c88d0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f57641c89e8>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f57e84303c8>)

In [6]:
print(np.shape(mnist.train.images))
print(np.shape(mnist.train.labels))

print(np.shape(mnist.validation.images))
print(np.shape(mnist.validation.labels))

print(np.shape(mnist.test.images))
print(np.shape(mnist.test.labels))

(55000, 784)
(55000, 10)
(5000, 784)
(5000, 10)
(10000, 784)
(10000, 10)


In [7]:
batch_size = 100
total_batch = mnist.train.num_examples // batch_size

In [8]:
total_batch

550

In [9]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

In [10]:
# 미니 배치 쓰는 이유
# 1. gpu문제
# 2. local optima 에 빠질 위험이 적다

In [14]:
W1 = tf.Variable(tf.random_normal([784,256], stddev=0.01))
B1 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L1 = tf.nn.sigmoid(tf.matmul(X,W1) + B1)

W2 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
B2 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))
model = tf.nn.softmax(tf.matmul(L1,W2)+B2)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(tf.clip_by_value(model, 1e-10, 1.0)), [1]))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [16]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(15):
        total_cost = 0
        total_acc = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val, acc_val = sess.run([optimizer, cost, accuracy],
                                           feed_dict={X:batch_xs, Y:batch_ys})
            total_cost += cost_val
            total_acc += acc_val
            
        print('Epoch: {0:04d}, Avg. cost = {1:.3f}, Train Acc. = {2}'.format(epoch + 1,
                                                                             total_cost / total_batch,
                                                                             total_acc / total_batch))
    print('Training Done!')
    print('Test Acc. = ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))


Epoch: 0001, Avg. cost = 0.606, Train Acc. = 0.8357636373896491
Epoch: 0002, Avg. cost = 0.236, Train Acc. = 0.9319636395844546
Epoch: 0003, Avg. cost = 0.185, Train Acc. = 0.9470545459877361
Epoch: 0004, Avg. cost = 0.146, Train Acc. = 0.95752727616917
Epoch: 0005, Avg. cost = 0.122, Train Acc. = 0.9645636412230405
Epoch: 0006, Avg. cost = 0.104, Train Acc. = 0.9709090987118808
Epoch: 0007, Avg. cost = 0.089, Train Acc. = 0.9750363729216835
Epoch: 0008, Avg. cost = 0.076, Train Acc. = 0.9791454658725045
Epoch: 0009, Avg. cost = 0.066, Train Acc. = 0.9814181928201156
Epoch: 0010, Avg. cost = 0.056, Train Acc. = 0.9850000104037198
Epoch: 0011, Avg. cost = 0.049, Train Acc. = 0.9867091008749875
Epoch: 0012, Avg. cost = 0.041, Train Acc. = 0.9891454631632025
Epoch: 0013, Avg. cost = 0.037, Train Acc. = 0.9901454624262723
Epoch: 0014, Avg. cost = 0.031, Train Acc. = 0.9924181886152788
Epoch: 0015, Avg. cost = 0.027, Train Acc. = 0.9942545509338379
Training Done!
Test Acc. =  0.9784


### --------------------------------------------------
# five layers

In [17]:
W1 = tf.Variable(tf.random_normal([784,1024], stddev=0.01))
B1 = tf.Variable(tf.random_normal(shape=[1024], stddev=0.01))
L1 = tf.nn.sigmoid(tf.matmul(X,W1) + B1)

W2 = tf.Variable(tf.random_normal([1024, 512], stddev=0.01))
B2 = tf.Variable(tf.random_normal(shape=[512], stddev=0.01))
L2 = tf.nn.sigmoid(tf.matmul(L1, W2) + B2)

W3 = tf.Variable(tf.random_normal([512,256], stddev=0.01))
B3 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L3 = tf.nn.sigmoid(tf.matmul(L2, W3) + B3)

W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
B4 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))
model = tf.nn.softmax(tf.matmul(L3,W4) + B4)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(tf.clip_by_value(model, 1e-10, 1.0)), [1]))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [18]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(15):
        total_cost = 0
        total_acc = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val, acc_val = sess.run([optimizer, cost, accuracy],
                                           feed_dict={X:batch_xs, Y:batch_ys})
            total_cost += cost_val
            total_acc += acc_val
            
        print('Epoch: {0:04d}, Avg. cost = {1:.3f}, Train Acc. = {2}'.format(epoch + 1,
                                                                             total_cost / total_batch,
                                                                             total_acc / total_batch))
    print('Training Done!')
    print('Test Acc. = ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001, Avg. cost = 0.899, Train Acc. = 0.6744727277349342
Epoch: 0002, Avg. cost = 0.203, Train Acc. = 0.9405272730914029
Epoch: 0003, Avg. cost = 0.131, Train Acc. = 0.9612545497850938
Epoch: 0004, Avg. cost = 0.099, Train Acc. = 0.9708545537428422
Epoch: 0005, Avg. cost = 0.078, Train Acc. = 0.9770727372169494
Epoch: 0006, Avg. cost = 0.060, Train Acc. = 0.9822000115567988
Epoch: 0007, Avg. cost = 0.050, Train Acc. = 0.9850000104037198
Epoch: 0008, Avg. cost = 0.041, Train Acc. = 0.9873818278312683
Epoch: 0009, Avg. cost = 0.031, Train Acc. = 0.9903818261623383
Epoch: 0010, Avg. cost = 0.028, Train Acc. = 0.9913272804563695
Epoch: 0011, Avg. cost = 0.021, Train Acc. = 0.9938363694060932
Epoch: 0012, Avg. cost = 0.017, Train Acc. = 0.994654550335624
Epoch: 0013, Avg. cost = 0.016, Train Acc. = 0.994963641166687
Epoch: 0014, Avg. cost = 0.017, Train Acc. = 0.9948363680189306
Epoch: 0015, Avg. cost = 0.013, Train Acc. = 0.996054549108852
Training Done!
Test Acc. =  0.9787


### --------------------------------------------------
# ten layers (extream case)

In [19]:
W1 = tf.Variable(tf.random_normal([784,1024], stddev=0.01))
B1 = tf.Variable(tf.random_normal(shape=[1024], stddev=0.01))
L1 = tf.nn.sigmoid(tf.matmul(X,W1) + B1)

W2 = tf.Variable(tf.random_normal([1024, 512], stddev=0.01))
B2 = tf.Variable(tf.random_normal(shape=[512], stddev=0.01))
L2 = tf.nn.sigmoid(tf.matmul(L1, W2) + B2)

W3 = tf.Variable(tf.random_normal([512,256], stddev=0.01))
B3 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L3 = tf.nn.sigmoid(tf.matmul(L2, W3) + B3)

W4 = tf.Variable(tf.random_normal([256,256], stddev=0.01))
B4 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L4 = tf.nn.sigmoid(tf.matmul(L3, W4) + B4)

W5 = tf.Variable(tf.random_normal([256,256], stddev=0.01))
B5 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L5 = tf.nn.sigmoid(tf.matmul(L4, W5) + B5)

W6 = tf.Variable(tf.random_normal([256,256], stddev=0.01))
B6 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L6 = tf.nn.sigmoid(tf.matmul(L5, W6) + B6)

W7 = tf.Variable(tf.random_normal([256,256], stddev=0.01))
B7 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L7 = tf.nn.sigmoid(tf.matmul(L6, W7) + B7)

W8 = tf.Variable(tf.random_normal([256,256], stddev=0.01))
B8 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L8 = tf.nn.sigmoid(tf.matmul(L7, W8) + B8)

W9 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
B9 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))
model = tf.nn.softmax(tf.matmul(L8,W9) + B9)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(tf.clip_by_value(model, 1e-10, 1.0)), [1]))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [20]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(15):
        total_cost = 0
        total_acc = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val, acc_val = sess.run([optimizer, cost, accuracy],
                                           feed_dict={X:batch_xs, Y:batch_ys})
            total_cost += cost_val
            total_acc += acc_val
            
        print('Epoch: {0:04d}, Avg. cost = {1:.3f}, Train Acc. = {2}'.format(epoch + 1,
                                                                             total_cost / total_batch,
                                                                             total_acc / total_batch))
    print('Training Done!')
    print('Test Acc. = ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001, Avg. cost = 2.302, Train Acc. = 0.11212727250361984
Epoch: 0002, Avg. cost = 2.233, Train Acc. = 0.12879999996924943
Epoch: 0003, Avg. cost = 1.983, Train Acc. = 0.20021818181330509
Epoch: 0004, Avg. cost = 1.940, Train Acc. = 0.20167272700504824
Epoch: 0005, Avg. cost = 1.888, Train Acc. = 0.2234909088774161
Epoch: 0006, Avg. cost = 1.421, Train Acc. = 0.41690908832983536
Epoch: 0007, Avg. cost = 1.065, Train Acc. = 0.5457090880654075
Epoch: 0008, Avg. cost = 0.779, Train Acc. = 0.6623272740299051
Epoch: 0009, Avg. cost = 0.661, Train Acc. = 0.7091454556855289
Epoch: 0010, Avg. cost = 0.544, Train Acc. = 0.7818727257034995
Epoch: 0011, Avg. cost = 0.431, Train Acc. = 0.8429272711277008
Epoch: 0012, Avg. cost = 0.389, Train Acc. = 0.8533818166906183
Epoch: 0013, Avg. cost = 0.346, Train Acc. = 0.8620545457709919
Epoch: 0014, Avg. cost = 0.335, Train Acc. = 0.865381817817688
Epoch: 0015, Avg. cost = 0.324, Train Acc. = 0.867999999631535
Training Done!
Test Acc. =  0.8605


### ---------------------------
# apply reLU at 5layers

In [24]:
W1 = tf.Variable(tf.random_normal([784,1024], stddev=0.01))
B1 = tf.Variable(tf.random_normal(shape=[1024], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X,W1) + B1)

W2 = tf.Variable(tf.random_normal([1024, 512], stddev=0.01))
B2 = tf.Variable(tf.random_normal(shape=[512], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2) + B2)

W3 = tf.Variable(tf.random_normal([512,256], stddev=0.01))
B3 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L3 = tf.nn.relu(tf.matmul(L2, W3) + B3)

W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
B4 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))
model = tf.nn.softmax(tf.matmul(L3,W4) + B4)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(tf.clip_by_value(model, 1e-10, 1.0)), [1]))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [25]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(15):
        total_cost = 0
        total_acc = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val, acc_val = sess.run([optimizer, cost, accuracy],
                                           feed_dict={X:batch_xs, Y:batch_ys})
            total_cost += cost_val
            total_acc += acc_val
            
        print('Epoch: {0:04d}, Avg. cost = {1:.3f}, Train Acc. = {2}'.format(epoch + 1,
                                                                             total_cost / total_batch,
                                                                             total_acc / total_batch))
    print('Training Done!')
    print('Test Acc. = ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001, Avg. cost = 0.354, Train Acc. = 0.8852727293968201
Epoch: 0002, Avg. cost = 0.116, Train Acc. = 0.9642727318677036
Epoch: 0003, Avg. cost = 0.078, Train Acc. = 0.9762181905182925
Epoch: 0004, Avg. cost = 0.056, Train Acc. = 0.982963646216826
Epoch: 0005, Avg. cost = 0.041, Train Acc. = 0.9869091002507643
Epoch: 0006, Avg. cost = 0.034, Train Acc. = 0.9892727355523543
Epoch: 0007, Avg. cost = 0.032, Train Acc. = 0.9897636447169564
Epoch: 0008, Avg. cost = 0.024, Train Acc. = 0.9921636430783706
Epoch: 0009, Avg. cost = 0.022, Train Acc. = 0.993200006051497
Epoch: 0010, Avg. cost = 0.022, Train Acc. = 0.9928000065413388
Epoch: 0011, Avg. cost = 0.015, Train Acc. = 0.9951272773742675
Epoch: 0012, Avg. cost = 0.017, Train Acc. = 0.9950181864608418
Epoch: 0013, Avg. cost = 0.014, Train Acc. = 0.995600004196167
Epoch: 0014, Avg. cost = 0.013, Train Acc. = 0.9960727309096943
Epoch: 0015, Avg. cost = 0.009, Train Acc. = 0.9973090934753418
Training Done!
Test Acc. =  0.9795


# --------------------
#apply dropout at 5layers

In [28]:
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([784,1024], stddev=0.01))
B1 = tf.Variable(tf.random_normal(shape=[1024], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X,W1) + B1)
L1 = tf.nn.dropout(L1, keep_prob)

W2 = tf.Variable(tf.random_normal([1024, 512], stddev=0.01))
B2 = tf.Variable(tf.random_normal(shape=[512], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1, W2) + B2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([512,256], stddev=0.01))
B3 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L3 = tf.nn.relu(tf.matmul(L2, W3) + B3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
B4 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))
model = tf.nn.softmax(tf.matmul(L3,W4) + B4)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(tf.clip_by_value(model, 1e-10, 1.0)), [1]))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [30]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(15):
        total_cost = 0
        total_acc = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val, acc_val = sess.run([optimizer, cost, accuracy],
                                           feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7})
            total_cost += cost_val
            total_acc += acc_val
            
        print('Epoch: {0:04d}, Avg. cost = {1:.3f}, Train Acc. = {2}'.format(epoch + 1,
                                                                             total_cost / total_batch,
                                                                             total_acc / total_batch))
    print('Training Done!')
    print('Test Acc. = ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0}))

Epoch: 0001, Avg. cost = 0.397, Train Acc. = 0.8740909094024788
Epoch: 0002, Avg. cost = 0.142, Train Acc. = 0.9575636390122501
Epoch: 0003, Avg. cost = 0.107, Train Acc. = 0.9685818259282546
Epoch: 0004, Avg. cost = 0.083, Train Acc. = 0.9748909187316894
Epoch: 0005, Avg. cost = 0.071, Train Acc. = 0.9780363730950788
Epoch: 0006, Avg. cost = 0.065, Train Acc. = 0.9801454650272022
Epoch: 0007, Avg. cost = 0.055, Train Acc. = 0.9831818282604218
Epoch: 0008, Avg. cost = 0.050, Train Acc. = 0.98452728303996
Epoch: 0009, Avg. cost = 0.043, Train Acc. = 0.9866545552557165
Epoch: 0010, Avg. cost = 0.041, Train Acc. = 0.9873818278312683
Epoch: 0011, Avg. cost = 0.039, Train Acc. = 0.9875818272070451
Epoch: 0012, Avg. cost = 0.038, Train Acc. = 0.9881818265264685
Epoch: 0013, Avg. cost = 0.033, Train Acc. = 0.9900909178907221
Epoch: 0014, Avg. cost = 0.033, Train Acc. = 0.9897272811152719
Epoch: 0015, Avg. cost = 0.029, Train Acc. = 0.9915818258849057
Training Done!
Test Acc. =  0.9823


# ----------------------
#apply batch normalization at 5layers

In [36]:
epsilon = 1e-3

W1 = tf.Variable(tf.random_normal([784,1024], stddev=0.01))
B1 = tf.Variable(tf.random_normal(shape=[1024], stddev=0.01))
L1_BN = tf.matmul(X,W1) + B1
batch_mean1, batch_var1 = tf.nn.moments(L1_BN, [0])
L1_hat = (L1_BN - batch_mean1)/tf.sqrt(batch_var1+epsilon)
scale1 = tf.Variable(tf.ones([1024]))
beta1 = tf.Variable(tf.ones([1024]))
L1 = tf.nn.relu(scale1*L1_hat + beta1)

W2 = tf.Variable(tf.random_normal([1024, 512], stddev=0.01))
B2 = tf.Variable(tf.random_normal(shape=[512], stddev=0.01))
L2_BN = tf.matmul(L1,W2) + B2
batch_mean2, batch_var2 = tf.nn.moments(L2_BN, [0])
L2_hat = (L2_BN - batch_mean2)/tf.sqrt(batch_var2+epsilon)
scale2 = tf.Variable(tf.ones([512]))
beta2 = tf.Variable(tf.ones([512]))
L2 = tf.nn.relu(scale2*L2_hat + beta2)

W3 = tf.Variable(tf.random_normal([512,256], stddev=0.01))
B3 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L3_BN = tf.matmul(L2,W3) + B3
batch_mean3, batch_var3 = tf.nn.moments(L3_BN, [0])
L3_hat = (L3_BN - batch_mean3)/tf.sqrt(batch_var3+epsilon)
scale3 = tf.Variable(tf.ones([256]))
beta3 = tf.Variable(tf.ones([256]))
L3 = tf.nn.relu(scale3*L3_hat + beta3)

W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
B4 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))
L4_BN = tf.matmul(L3,W4) + B4
batch_mean4, batch_var4 = tf.nn.moments(L4_BN, [0])
L4_hat = (L4_BN - batch_mean4)/tf.sqrt(batch_var4+epsilon)
scale4 = tf.Variable(tf.ones([10]))
beta4 = tf.Variable(tf.ones([10]))
model = tf.nn.softmax(scale4*L4_hat + beta4)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(tf.clip_by_value(model, 1e-10, 1.0)), [1]))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [37]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(15):
        total_cost = 0
        total_acc = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val, acc_val = sess.run([optimizer, cost, accuracy],
                                           feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7})
            total_cost += cost_val
            total_acc += acc_val
            
        print('Epoch: {0:04d}, Avg. cost = {1:.3f}, Train Acc. = {2}'.format(epoch + 1,
                                                                             total_cost / total_batch,
                                                                             total_acc / total_batch))
    print('Training Done!')
    print('Test Acc. = ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0}))

Epoch: 0001, Avg. cost = 0.438, Train Acc. = 0.9311454553224824
Epoch: 0002, Avg. cost = 0.209, Train Acc. = 0.9634000053189018
Epoch: 0003, Avg. cost = 0.141, Train Acc. = 0.9726000081409107
Epoch: 0004, Avg. cost = 0.102, Train Acc. = 0.9781091012737968
Epoch: 0005, Avg. cost = 0.082, Train Acc. = 0.9811818285421892
Epoch: 0006, Avg. cost = 0.066, Train Acc. = 0.9845454649491744
Epoch: 0007, Avg. cost = 0.056, Train Acc. = 0.9862363725358789
Epoch: 0008, Avg. cost = 0.045, Train Acc. = 0.9894363727352836
Epoch: 0009, Avg. cost = 0.041, Train Acc. = 0.9890909176523035
Epoch: 0010, Avg. cost = 0.036, Train Acc. = 0.9909090987118808
Epoch: 0011, Avg. cost = 0.032, Train Acc. = 0.9916181887279857
Epoch: 0012, Avg. cost = 0.027, Train Acc. = 0.99249091592702
Epoch: 0013, Avg. cost = 0.026, Train Acc. = 0.9927818247404966
Epoch: 0014, Avg. cost = 0.022, Train Acc. = 0.9938727327910336
Epoch: 0015, Avg. cost = 0.022, Train Acc. = 0.9938727323575454
Training Done!
Test Acc. =  0.9814


# -------------------------
####apply dropout + batch normalization at 5layers

In [40]:
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([784,1024], stddev=0.01))
B1 = tf.Variable(tf.random_normal(shape=[1024], stddev=0.01))
L1_BN = tf.matmul(X,W1) + B1
batch_mean1, batch_var1 = tf.nn.moments(L1_BN, [0])
L1_hat = (L1_BN - batch_mean1)/tf.sqrt(batch_var1+epsilon)
scale1 = tf.Variable(tf.ones([1024]))
beta1 = tf.Variable(tf.ones([1024]))
L1 = tf.nn.sigmoid(scale1*L1_hat + beta1)
L1 = tf.nn.dropout(L1, keep_prob)

W2 = tf.Variable(tf.random_normal([1024, 512], stddev=0.01))
B2 = tf.Variable(tf.random_normal(shape=[512], stddev=0.01))
L2 = tf.nn.sigmoid(tf.matmul(L1, W2) + B2)
L2_BN = tf.matmul(L1,W2) + B2
batch_mean2, batch_var2 = tf.nn.moments(L2_BN, [0])
L2_hat = (L2_BN - batch_mean2)/tf.sqrt(batch_var2+epsilon)
scale2 = tf.Variable(tf.ones([512]))
beta2 = tf.Variable(tf.ones([512]))
L2 = tf.nn.sigmoid(scale2*L2_hat + beta2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([512,256], stddev=0.01))
B3 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L3_BN = tf.matmul(L2,W3) + B3
batch_mean3, batch_var3 = tf.nn.moments(L3_BN, [0])
L3_hat = (L3_BN - batch_mean3)/tf.sqrt(batch_var3+epsilon)
scale3 = tf.Variable(tf.ones([256]))
beta3 = tf.Variable(tf.ones([256]))
L3 = tf.nn.sigmoid(scale3*L3_hat + beta3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
B4 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))
L4_BN = tf.matmul(L3,W4) + B4
batch_mean4, batch_var4 = tf.nn.moments(L4_BN, [0])
L4_hat = (L4_BN - batch_mean4)/tf.sqrt(batch_var4+epsilon)
scale4 = tf.Variable(tf.ones([10]))
beta4 = tf.Variable(tf.ones([10]))
model = tf.nn.softmax(scale4*L4_hat + beta4)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(tf.clip_by_value(model, 1e-10, 1.0)), [1]))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [41]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(15):
        total_cost = 0
        total_acc = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val, acc_val = sess.run([optimizer, cost, accuracy],
                                           feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7})
            total_cost += cost_val
            total_acc += acc_val
            
        print('Epoch: {0:04d}, Avg. cost = {1:.3f}, Train Acc. = {2}'.format(epoch + 1,
                                                                             total_cost / total_batch,
                                                                             total_acc / total_batch))
    print('Training Done!')
    print('Test Acc. = ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0}))

Epoch: 0001, Avg. cost = 0.769, Train Acc. = 0.8313636355102062
Epoch: 0002, Avg. cost = 0.467, Train Acc. = 0.8932909114794297
Epoch: 0003, Avg. cost = 0.346, Train Acc. = 0.9154000020027161
Epoch: 0004, Avg. cost = 0.291, Train Acc. = 0.9237272757833654
Epoch: 0005, Avg. cost = 0.247, Train Acc. = 0.9353818199851296
Epoch: 0006, Avg. cost = 0.225, Train Acc. = 0.9376181833310561
Epoch: 0007, Avg. cost = 0.200, Train Acc. = 0.9442363650148565
Epoch: 0008, Avg. cost = 0.177, Train Acc. = 0.9504909119822762
Epoch: 0009, Avg. cost = 0.169, Train Acc. = 0.9515636385570873
Epoch: 0010, Avg. cost = 0.156, Train Acc. = 0.9556363666057587
Epoch: 0011, Avg. cost = 0.147, Train Acc. = 0.9580000022324648
Epoch: 0012, Avg. cost = 0.139, Train Acc. = 0.9591454582864588
Epoch: 0013, Avg. cost = 0.130, Train Acc. = 0.9619272767413746
Epoch: 0014, Avg. cost = 0.119, Train Acc. = 0.9649090976064856
Epoch: 0015, Avg. cost = 0.118, Train Acc. = 0.9650181883031672
Training Done!
Test Acc. =  0.9787


In [ ]:
#-----------------------------
#apply dropout + batch normalization + ReLU at 5layers

In [42]:
keep_prob = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([784,1024], stddev=0.01))
B1 = tf.Variable(tf.random_normal(shape=[1024], stddev=0.01))
L1_BN = tf.matmul(X,W1) + B1
batch_mean1, batch_var1 = tf.nn.moments(L1_BN, [0])
L1_hat = (L1_BN - batch_mean1)/tf.sqrt(batch_var1+epsilon)
scale1 = tf.Variable(tf.ones([1024]))
beta1 = tf.Variable(tf.ones([1024]))
L1 = tf.nn.relu(scale1*L1_hat + beta1)
L1 = tf.nn.dropout(L1, keep_prob)

W2 = tf.Variable(tf.random_normal([1024, 512], stddev=0.01))
B2 = tf.Variable(tf.random_normal(shape=[512], stddev=0.01))
L2 = tf.nn.sigmoid(tf.matmul(L1, W2) + B2)
L2_BN = tf.matmul(L1,W2) + B2
batch_mean2, batch_var2 = tf.nn.moments(L2_BN, [0])
L2_hat = (L2_BN - batch_mean2)/tf.sqrt(batch_var2+epsilon)
scale2 = tf.Variable(tf.ones([512]))
beta2 = tf.Variable(tf.ones([512]))
L2 = tf.nn.relu(scale2*L2_hat + beta2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([512,256], stddev=0.01))
B3 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L3_BN = tf.matmul(L2,W3) + B3
batch_mean3, batch_var3 = tf.nn.moments(L3_BN, [0])
L3_hat = (L3_BN - batch_mean3)/tf.sqrt(batch_var3+epsilon)
scale3 = tf.Variable(tf.ones([256]))
beta3 = tf.Variable(tf.ones([256]))
L3 = tf.nn.relu(scale3*L3_hat + beta3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256,10], stddev=0.01))
B4 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))
L4_BN = tf.matmul(L3,W4) + B4
batch_mean4, batch_var4 = tf.nn.moments(L4_BN, [0])
L4_hat = (L4_BN - batch_mean4)/tf.sqrt(batch_var4+epsilon)
scale4 = tf.Variable(tf.ones([10]))
beta4 = tf.Variable(tf.ones([10]))
model = tf.nn.softmax(scale4*L4_hat + beta4)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(tf.clip_by_value(model, 1e-10, 1.0)), [1]))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [43]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(15):
        total_cost = 0
        total_acc = 0
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            _, cost_val, acc_val = sess.run([optimizer, cost, accuracy],
                                           feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7})
            total_cost += cost_val
            total_acc += acc_val
            
        print('Epoch: {0:04d}, Avg. cost = {1:.3f}, Train Acc. = {2}'.format(epoch + 1,
                                                                             total_cost / total_batch,
                                                                             total_acc / total_batch))
    print('Training Done!')
    print('Test Acc. = ', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0}))

Epoch: 0001, Avg. cost = 0.571, Train Acc. = 0.8936000008610162
Epoch: 0002, Avg. cost = 0.294, Train Acc. = 0.9418909095634114
Epoch: 0003, Avg. cost = 0.207, Train Acc. = 0.9548545479774475
Epoch: 0004, Avg. cost = 0.157, Train Acc. = 0.9637454591014168
Epoch: 0005, Avg. cost = 0.131, Train Acc. = 0.9684545537558469
Epoch: 0006, Avg. cost = 0.111, Train Acc. = 0.9721272820776159
Epoch: 0007, Avg. cost = 0.096, Train Acc. = 0.9742363720590418
Epoch: 0008, Avg. cost = 0.087, Train Acc. = 0.9771636466546492
Epoch: 0009, Avg. cost = 0.075, Train Acc. = 0.9799272829836065
Epoch: 0010, Avg. cost = 0.068, Train Acc. = 0.9810909200798381
Epoch: 0011, Avg. cost = 0.061, Train Acc. = 0.9830000112273476
Epoch: 0012, Avg. cost = 0.060, Train Acc. = 0.9827272831309926
Epoch: 0013, Avg. cost = 0.050, Train Acc. = 0.9864363740790975
Epoch: 0014, Avg. cost = 0.052, Train Acc. = 0.9852545560490001
Epoch: 0015, Avg. cost = 0.044, Train Acc. = 0.9876363730430603
Training Done!
Test Acc. =  0.9838
